# Introduction to Building LLM Agents with Tools and Tracing

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Running the agent on a multi-step task.

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!pip install weave requests "mistralai[gcp]>=1.0.0"
```


In [93]:
# Global Configuration & Setup
import inspect
import json
import os
import requests
import weave # Must import weave before litellm for auto-patching
from enum import Enum
from pydantic import BaseModel, Field
from rich.pretty import pprint
from typing import Any, Callable, Dict, List, get_type_hints

# from mistralai_gcp import MistralGoogleCloud
from mistralai import Mistral

Define a model to use, as we are going to use tool calling you need a capable model like `mistral-large`

In [18]:

# Set the model name you want to use globally.

MISTRAL_LARGE_MODEL = "mistral-large-2411" # vertex name for the model
MISTRAL_SMALL_MODEL = "mistral-small-2503"

client = Mistral(api_key=os.environ.get("MISTRAL_API_KEY"))

# region = os.environ.get("GOOGLE_CLOUD_REGION")
# project_id = os.environ.get("GOOGLE_CLOUD_PROJECT_ID")
# client = MistralGoogleCloud(region=region, project_id=project_id, timeout_ms=120_000)

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [19]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init('intro-to-agents-script')

## 1. Basic LLM Call with LiteLLM

Let's start with a simple call to the LLM using/

![](images/01_trace.png)

In [21]:
# Define a simple message list (conversation history)
messages = [{"role": "user", "content": "Hello, LLM! How does an AI agent work?"}]

# Make the call
response = resp = client.chat.complete(
    model = MISTRAL_SMALL_MODEL,
    messages=messages,
)
# Print the response content
assistant_response = response.choices[0].message.content
print(f"LLM Response:\\n{assistant_response}")

# Click on the 🍩 linke below to see the trace in Weave 👇

LLM Response:\nHello! An AI agent is a software entity that can perceive its environment and take actions to achieve its goals. Here's a simplified breakdown of how an AI agent typically works:

### 1. **Perception**
   - **Sensors**: The agent uses sensors to gather information from its environment. These sensors can be physical (like cameras, microphones, or touch sensors) or virtual (like data feeds, APIs, or user inputs).
   - **Data Processing**: The raw data from the sensors is processed to extract meaningful information. This might involve filtering, normalization, or other forms of data preprocessing.

### 2. **Knowledge Representation**
   - **Models**: The agent uses models to represent the information it has gathered. These models can be simple (like rules or decision trees) or complex (like neural networks).
   - **State Representation**: The agent maintains a representation of the current state of the environment and its own state.

### 3. **Decision Making**
   - **Goal S

Because we imported `weave` before `litellm` and called `weave.init()`, the `litellm.completion` call above was automatically traced. You can open your Weave dashboard and navigate to the `intro-to-agents-script` project to see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [40]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.

@weave.op
def call_model(model_name: str, messages: List[Dict[str, Any]], **kwargs) -> str:
    "Call a model with the given messages and kwargs."
    response = client.chat.complete(
        model=model_name,
        messages=messages,
        **kwargs
    )

    return response.choices[0].message

response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages)
print(response.content)

To find the sum of your lucky numbers 77 and 11, you simply add them together:

77 + 11 = 88

So, the sum of your lucky numbers is 88.



## 2. Introducing Tool Calling (Manual Definition)

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [41]:
@weave.op 
def add_numbers(a: int, b: int) -> int:
    """Adds two numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [42]:
add_numbers(1, 2)

3

In [43]:
# this doesn't work...
call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages, tools=[add_numbers])

ValidationError: 2 validation errors for Unmarshaller
body.nullable[list[Tool]].0
  Input should be a valid dictionary or instance of Tool [type=model_type, input_value=<function add_numbers at 0x169f8d6c0>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
body.Unset
  Input should be a valid dictionary or instance of Unset [type=model_type, input_value=[<function add_numbers at 0x169f8d6c0>], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

Next, we manually create the JSON schema describing this tool in a format that models like GPT or Mistral understand.

In [44]:
# Manually define the tool schema
tool_add_numbers_schema = {
    "type": "function",
    "function": {
        "name": "add_numbers",
        "description": "Adds two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "The first number."
                },
                "b": {
                    "type": "integer",
                    "description": "The second number."
                }
            },
            "required": ["a", "b"]
        }
    }
}

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [82]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that can use tools to answer questions."},
    {"role": "user", "content": "My lucky numbers are 77 and 11. What is their sum?"}]
response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages, tools=[tool_add_numbers_schema])
pprint(response)

AssistantMessage(
│   content='',
│   tool_calls=[
│   │   ToolCall(
│   │   │   function=FunctionCall(name='add_numbers', arguments='{"a": 77, "b": 11}'),
│   │   │   id='gamS50kKt',
│   │   │   type=None,
│   │   │   index=0
│   │   )
│   ],
│   prefix=False,
│   role='assistant'
)

## Manual Tool Call
The LLM's response might contain a request to call our tool (`response.choices[0].message.tool_calls`) or it might respond directly (`response.choices[0].message.content`). If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [83]:
if response.tool_calls:
    print("LLM requested a tool call:")
    for tool_call in response.tool_calls:
        function_name = tool_call.function.name
        function_args_str = tool_call.function.arguments
        function_args = json.loads(function_args_str)
        print(f"  - Tool: {function_name}, Args: {function_args_str}")
        if function_name == "add_numbers":
            tool_result_content = add_numbers(**function_args)

print(tool_result_content)

LLM requested a tool call:
  - Tool: add_numbers, Args: {"a": 77, "b": 11}
88


We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [84]:
messages.append(response.model_dump())

In [85]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": str(tool_result_content)
})

In [86]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant that can use tools to answer questions.'},
 {'role': 'user',
  'content': 'My lucky numbers are 77 and 11. What is their sum?'},
 {'content': '',
  'tool_calls': [{'function': {'name': 'add_numbers',
     'arguments': '{"a": 77, "b": 11}'},
    'id': 'gamS50kKt',
    'type': None,
    'index': 0}],
  'prefix': False,
  'role': 'assistant'},
 {'tool_call_id': 'gamS50kKt',
  'role': 'tool',
  'name': 'add_numbers',
  'content': '88'}]

Now call the model again with the new messages and it will use the tool call result to answer the question

In [88]:
final_response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages)
print(final_response.content)

The sum of your lucky numbers 77 and 11 is 88.


## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [89]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a tool-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": inspect.getdoc(func).split("\\n")[0] if inspect.getdoc(func) else "",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }

    docstring = inspect.getdoc(func)
    param_descriptions = {}
    if docstring:
        args_section = False
        current_param = None
        for line in docstring.split('\\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped:
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" "): # Heuristic: end of args section
                     args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle List[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by List or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: OpenAI schema doesn't officially use default, but useful metadata

        schema["function"]["parameters"]["properties"][name] = param_schema
        if is_required:
            schema["function"]["parameters"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [95]:
tool_schema = generate_tool_schema(add_numbers)
pprint(tool_schema)

{
│   'type': 'function',
│   'function': {
│   │   'name': 'add_numbers',
│   │   'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
│   │   'parameters': {
│   │   │   'type': 'object',
│   │   │   'properties': {
│   │   │   │   'a': {'type': 'integer', 'description': ''},
│   │   │   │   'b': {'type': 'integer', 'description': ''}
│   │   │   },
│   │   │   'required': ['a', 'b']
│   │   }
│   }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [97]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [98]:
add_numbers = function_tool(add_numbers)
pprint(add_numbers.tool_schema)


{
│   'type': 'function',
│   'function': {
│   │   'name': 'add_numbers',
│   │   'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
│   │   'parameters': {
│   │   │   'type': 'object',
│   │   │   'properties': {
│   │   │   │   'a': {'type': 'integer', 'description': ''},
│   │   │   │   'b': {'type': 'integer', 'description': ''}
│   │   │   },
│   │   │   'required': ['a', 'b']
│   │   }
│   }
}

In [99]:
add_numbers.tool_schema

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': ''},
    'b': {'type': 'integer', 'description': ''}},
   'required': ['a', 'b']}}}

and call the tool =)

In [101]:
add_numbers(1, 2)

3

Let's create a real example hitting an API: The pokemon API

In [107]:
@weave.op 
@function_tool
def get_pokemon_info(pokemon_name: str) -> str:
    """Fetches minimal information (name, ID, weight) for a specific Pokemon. Weight is in hectograms.

    Args:
        pokemon_name: The name or Pokedex ID of the Pokemon.
    """
    base_url = "https://pokeapi.co/api/v2/pokemon/"
    try:
        response = requests.get(f"{base_url}{pokemon_name.lower().strip()}")
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        info = {
            "name": data.get('name', 'Unknown').capitalize(),
            "id": data.get('id', -1),
            "weight": data.get('weight', -1) # Weight in hectograms
        }
        return info
    except Exception as e:
        return {"error": f"Error fetching pokemon {pokemon_name}: {str(e)}"}

In [108]:
get_pokemon_info("pikachu")

{'name': 'Pikachu', 'id': 25, 'weight': 60}

In [109]:
get_pokemon_info.tool_schema

{'type': 'function',
 'function': {'name': 'get_pokemon_info',
  'description': 'Fetches minimal information (name, ID, weight) for a specific Pokemon. Weight is in hectograms.\n\nArgs:\n    pokemon_name: The name or Pokedex ID of the Pokemon.',
  'parameters': {'type': 'object',
   'properties': {'pokemon_name': {'type': 'string', 'description': ''}},
   'required': ['pokemon_name']}}}

In [117]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that can use tools to answer questions."},
    {"role": "user", "content": "What is the weight of Pikachu?"}]

response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages, tools=[get_pokemon_info.tool_schema])
pprint(response)

AssistantMessage(
│   content='',
│   tool_calls=[
│   │   ToolCall(
│   │   │   function=FunctionCall(name='get_pokemon_info', arguments='{"pokemon_name": "Pikachu"}'),
│   │   │   id='8hu2SIoBS',
│   │   │   type=None,
│   │   │   index=0
│   │   )
│   ],
│   prefix=False,
│   role='assistant'
)

In [119]:
from mistralai import ToolCall

def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], tool_calls: list[ToolCall]) -> list[dict]:
    "Perform the tool calls and return the messages with the tool call results"
    messages = []
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        tool = get_tool(tools, function_name)
        tool_response = tool(**function_args)
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "content": str(tool_response),
        })
    return messages

# add the tool call result to the messages
messages.append(response.model_dump())
messages.extend(perform_tool_calls(tools=[get_pokemon_info], tool_calls=response.tool_calls))

final_response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages)
print(final_response.content)


The weight of Pikachu is 60 pounds.


Let's wrap this in a function and add a few more tools.

In [124]:
@weave.op
def pokedex(pokemon_question: str) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant that can use tools to answer questions."},
        {"role": "user", "content": pokemon_question}]

    response = call_model(
        model_name=MISTRAL_SMALL_MODEL, 
        messages=messages, 
        tools=[get_pokemon_info.tool_schema, 
            add_numbers.tool_schema])
    
    messages.append(response.model_dump())
    messages.extend(perform_tool_calls(tools=[get_pokemon_info], tool_calls=response.tool_calls))

    final_response = call_model(model_name=MISTRAL_SMALL_MODEL, messages=messages)
    return final_response.content

In [125]:
pokedex("What is the combined weight of Ash's first 3 pokemons?")

"The combined weight of Ash's first 3 pokemons is 409."

This is "Almost" an agent, but it's missing the loop. Let's add that next.